[View in Colaboratory](https://colab.research.google.com/github/pkraison/ex.ai/blob/master/Machine_learning_Cython.ipynb)

In [41]:
!pip install cython
!pip install spacy

In [0]:
from random import random

class Rectangle:
  def __init__(self,h,w):
    self.h = h
    self.w = w
   
  def area(self):
    return self.h*self.w


In [0]:
def check_rectangles(rectangles, threshold):
    n_out = 0
    for r in rectangles:
      if r.area() > threshold:
        n_out += 1
    return n_out 

In [0]:
def main_rectangles_slow():
    n_rectangles = 1000000
    rectangles = list(Rectangle(random(), random()) for i in range(n_rectangles))
    n_out = check_rectangles(rectangles, threshold = 0.25)
    print(n_out)

In [45]:
%%time
main_rectangles_slow()

404746
CPU times: user 2.33 s, sys: 272 ms, total: 2.6 s
Wall time: 2.6 s


In [0]:
%reload_ext Cython

In [0]:
%%cython
from cymem.cymem cimport Pool
from random import random
cdef struct Rectangle:
  float w
  float h
  
  
cdef int check_rectangles_cy(Rectangle* rectangles, int n_rectangles, float threshold):
  cdef int n_out = 0
  for rectangle in rectangles[:n_rectangles]:
    if rectangle.w*rectangle.h > threshold:
      n_out += 1
  return n_out

def main_rectangles_fast():
  cdef int n_rectangles = 1000000
  cdef float threshold = 0.25
  
  cdef Pool mem = Pool()
  cdef Rectangle* rectangles = <Rectangle*>mem.alloc(n_rectangles, sizeof(Rectangle))

  for i in range(n_rectangles):
    rectangles[i].w = random()
    rectangles[i].h = random()
  n_out = check_rectangles_cy(rectangles, n_rectangles, threshold)
  print(n_out)

In [52]:
%%time 
main_rectangles_fast()

403809
CPU times: user 83.2 ms, sys: 1.95 ms, total: 85.2 ms
Wall time: 84.5 ms


In [53]:
import spacy.cli
spacy.cli.download("en")
nlp = spacy.load('en')



# Test Data
multiSentence = "There is an art, it says, or rather, a knack to flying." \
                 "The knack lies in learning how to throw yourself at the ground and miss." \
                 "In the beginning the Universe was created. This has made a lot of people "\
                 "very angry and been widely regarded as a bad move."
parsedData = nlp(multiSentence)


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [54]:
for span in parsedData.sents:
    sent = [parsedData[i] for i in range(span.start, span.end)]
    break

for token in sent:
    print(token.orth_, token.pos_)

There ADV
is VERB
an DET
art NOUN
, PUNCT
it PRON
says VERB
, PUNCT
or CCONJ
rather ADV
, PUNCT
a DET
knack NOUN
to ADP
flying NOUN
. PUNCT


In [55]:
# Let's look at the named entities of this example:
example = "Apple's stocks dropped dramatically after the death of Steve Jobs in October."
parsedEx = nlp(example)
for token in parsedEx:
    print(token.orth_, token.ent_type_ if token.ent_type_ != "" else "(not an entity)")

print("-------------- entities only ---------------")
# if you just want the entities and nothing else, you can do access the parsed examples "ents" property like this:
ents = list(parsedEx.ents)
for entity in ents:
    print(entity.label, entity.label_, ' '.join(t.orth_ for t in entity))

Apple ORG
's (not an entity)
stocks (not an entity)
dropped (not an entity)
dramatically (not an entity)
after (not an entity)
the (not an entity)
death (not an entity)
of (not an entity)
Steve PERSON
Jobs PERSON
in (not an entity)
October DATE
. (not an entity)
-------------- entities only ---------------
381 ORG Apple
378 PERSON Steve Jobs
388 DATE October
